In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp/matplotlib"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import time
import datetime
import sys
import psutil
import subprocess
import GPUtil

from scripts.data_loader import load_and_tokenize_data

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, Bidirectional, Dropout

from tensorflow.keras.callbacks import TensorBoard

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    classification_report, confusion_matrix
)
import wandb
from wandb.keras import WandbCallback

# Enable script imports
sys.path.append(os.path.abspath(".."))
from scripts.result_logger import log_results

2025-04-20 11:42:53.094302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745174573.116387  375057 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745174573.122721  375057 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745174573.139553  375057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745174573.139588  375057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745174573.139590  375057 computation_placer.cc:177] computation placer alr

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load data
X_train_seq = np.load("data/splits/X_train_seq.npy")
X_val_seq = np.load("data/splits/X_val_seq.npy")
X_test_seq = np.load("data/splits/X_test_seq.npy")
y_train_bin = np.load("data/splits/y_train_bin.npy")
y_val_bin = np.load("data/splits/y_val_bin.npy")
y_test_bin = np.load("data/splits/y_test_bin.npy")

# Best Parameters                                         #TODO
vocab_size = 10000
max_length = 100
embedding_dim = 64
batch_size = 64
epochs = 20

# CNN

In [ ]:
# wandb Init

wandb.init(
    project="sentiment-analysis", 
    name="CNN_Social", 
    config={
        "model": "CNN",
        "dataset": "Sentiment140",
        "max_length": max_length,
        "vocab_size": vocab_size,
        "embedding_dim": embedding_dim,
        "epochs": epochs,
        "batch_size": batch_size
})


In [ ]:
# Tensorboard logging

log_dir = f"logs/{wandb.run.name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Build CNN model

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),              #TODO
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),                   #TODO
    Dropout(0.5),
    Dense(1, activation='sigmoid')                   #TODO
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])                   #TODO


In [ ]:
# Train CNN model

start_time = time.time()

history = model.fit(
    X_train_seq, y_train_bin,
    validation_data=(X_val_seq, y_val_bin),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[tensorboard_callback, WandbCallback()],
    verbose=1
)

train_time = time.time() - start_time

In [ ]:
# Validation Evaluation
y_val_preds_cnn = (model.predict(X_val_seq) > 0.5).astype("int32")

print("\n CNN Model - Validation Set:")
print(classification_report(y_val_bin, y_val_preds_cnn))

val_acc = accuracy_score(y_val_bin, y_val_preds_cnn)
val_prec = precision_score(y_val_bin, y_val_preds_cnn)
val_rec = recall_score(y_val_bin, y_val_preds_cnn)
val_f1 = f1_score(y_val_bin, y_val_preds_cnn)

In [ ]:
# Test Evaluation
y_test_preds_cnn = (model.predict(X_test_seq) > 0.5).astype("int32").flatten()

print("\n CNN Model - Test Set:")
print(classification_report(y_test_bin, y_test_preds_cnn))

acc = accuracy_score(y_test_bin, y_test_preds_cnn)
prec = precision_score(y_test_bin, y_test_preds_cnn)
rec = recall_score(y_test_bin, y_test_preds_cnn)
f1 = f1_score(y_test_bin, y_test_preds_cnn)


In [ ]:
# Inference time 
start_infer = time.time()
test_preds = (model.predict(X_test_seq) > 0.5).astype("int32")
infer_time = (time.time() - start_infer) / len(X_test_seq)  # avg per sample

In [ ]:
# Confusion Matrix

cm = confusion_matrix(y_test_bin, y_test_preds_cnn, labels=[0, 1])
plt.figure(figsize=(4, 3))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title("Confusion Matrix - CNN Model")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig("cnn_confusion_matrix.png")

In [ ]:
# Systems Stats

cpu_usage = psutil.cpu_percent(interval=1)
mem_usage = psutil.virtual_memory().percent
gpu = GPUtil.getGPUs()[0]
power = subprocess.check_output([
    "nvidia-smi", "--query-gpu=power.draw", "--format=csv,noheader,nounits"
]).decode("utf-8").strip()

In [ ]:
# Log results to  result_logger.py & wandb Logs

log_results(
    model_name="CNN",
    dataset_name="Social Media",
    accuracy=acc,
    precision=prec,
    recall=rec,
    f1=f1,
    tuning_time=None,
    train_time=train_time,
    inference_time=infer_time,
    latency=infer_time * 1000,
    cpu_usage=cpu_usage,
    mem_usage=mem_usage,
    gpu_usage=gpu.load * 100,
    gpu_mem_usage=gpu.memoryUtil * 100,
    gpu_temp=gpu.temperature,
    power=float(gpu.powerUsage),
    features=vocab_size,
    hyperparams=f"embed_dim={embedding_dim}, epochs={epochs}, batch={batch_size}"
)

wandb.log({
    "Validation Accuracy": val_acc, 
    "Validation Precision": val_prec,
    "Validation Recall": val_rec, 
    "Validation F1 Score": val_f1,

    "Test Accuracy": acc,
    "Test Precision": prec,
    "Test Recall": rec,
    "Test F1": f1,

    "Training Time (s)": train_time,
    "Inference Time (s)": infer_time,
    "Latency (ms)": infer_time * 1000,

    "CPU Usage (%)": cpu_usage,
    "Memory Usage (%)": mem_usage,
    "GPU Load (%)": gpu.load * 100,
    "GPU Memory (%)": gpu.memoryUtil * 100,
    "GPU Temp": gpu.temperature,
    "Power Consumption (W)": float(power),

    "Confusion Matrix": wandb.Image("cnn_confusion_matrix.png")
})
wandb.finish()


# BiLSTM

In [ ]:
#BiLSTM

model = Sequential([
    Embedding(max_vocab, 128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
